In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
def predict_face(reshaped_face, model):
    CATEGORIES = ['real', 'fake']
    prediction = model.predict(reshaped_face)
    return CATEGORIES[int(prediction[0][0])]

In [3]:
def reshape_face(face):
    IMG_SIZE = 100
    img_array = cv2.imread(face) 
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [4]:
def get_face(image):
    file_name = 'haarcascade_frontalface_alt.xml'
    lib_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\venv\Lib\site-packages\cv2\data"
    face_cascade_path  = os.path.join(lib_dir_path, file_name)
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    face_exist = False
    for (x,y,w,h) in faces:
        face_exist = True
        image = image[y:y+h, x:x+w]
    if face_exist:
        return image
    else:
        return None

In [5]:
def predict_video(source_video_path, model):
    fake_frames = 0
    real_frames = 0
    vidcap = cv2.VideoCapture(source_video_path)
    success, image = vidcap.read()
    while success:
        ret, frame = vidcap.read()
        if frame is not None and ret is True:
            face = get_face(frame)
            if face is not None and face.shape[0] > 0 and face.shape[1]>0:
#                 plt.imshow(face)
#                 plt.show()
                cv2.imwrite('temporary.png', face)
                reshaped_face = reshape_face('temporary.png')
                result = predict_face(reshaped_face, model)
                if result == "fake":
                    fake_frames += 1
                else:
                    real_frames += 1
        else: 
            break
    print("To jest fake na {} %".format(fake_frames/(fake_frames+real_frames)))

In [6]:
model = tf.keras.models.load_model('mesonet.model')

In [7]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fakeExample1.mp4"
predict_video(source_video_path, model)

To jest fake na 0.571917808219178 %


In [15]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fakeExample2.mp4"
predict_video(source_video_path, model)

To jest fake na 0.5596330275229358 %


In [16]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fakeExample3.mp4"
predict_video(source_video_path, model)

To jest fake na 0.02766798418972332 %


In [17]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fakeExample4.mp4"
predict_video(source_video_path, model)

To jest fake na 0.008968609865470852 %


In [18]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\fakeExample5.mp4"
predict_video(source_video_path, model)

To jest fake na 0.47058823529411764 %


In [8]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_videos\abqwwspghj.mp4"
predict_video(source_video_path, model)

To jest fake na 0.14035087719298245 %
